## 문제
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler

## llm
llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

## 예시 제공
chat_examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question" : "what was last movie i asked?",
        "answer" : "대부"
    },
    {
        "question" : "what was last movie i asked?",
        "answer" : "탑건"
    }
]

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=chat_prompt,
    examples=chat_examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
          "You are a global movie expert. Your responses should adhere to the following rules: "
            "1. If the human provides only the name of a movie, respond with exactly three emojis that are relevant to the movie's themes, genre, or notable elements. "
            "2. If the human asks, 'What was the last movie I asked?', reply only with the title of the most recently mentioned movie by the human. "
            "3. Keep your responses brief and strictly aligned with the rules above, without adding any extra information or commentary."
        ),
        chat_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(input):
    print(input)
    ## 메모리 로드
    w = memory.load_memory_variables({})['history']
    print(w)
    return w

chain = RunnablePassthrough.assign(history=load_memory) | prompt  | llm

def invoke_chain(question):
    result = chain.invoke({
        "question" : question
    })
    ## 메모리에 저장
    memory.save_context({"input" : question}, {"output":result.content})


invoke_chain('love actually')
invoke_chain('titanic')

{'question': 'love actually'}
[]
❤️🎄👫{'question': 'titanic'}
[HumanMessage(content='love actually'), AIMessage(content='❤️🎄👫')]
🚢💔🎶

In [51]:
invoke_chain('최근에 물어본 영화 이름이 뭐야?')

{'question': '최근에 물어본 영화 이름이 뭐야?'}
[HumanMessage(content='love actually'), AIMessage(content='❤️🎄👫'), HumanMessage(content='titanic'), AIMessage(content='🚢💔🎶')]
titanic